<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Space X  Falcon 9 First Stage Landing Prediction**


## Web scraping Falcon 9 and Falcon Heavy Launches Records from Wikipedia


Estimated time needed: **40** minutes


In this lab, you will be performing web scraping to collect Falcon 9 historical launch records from a Wikipedia page titled `List of Falcon 9 and Falcon Heavy launches`

https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/Falcon9_rocket_family.svg)


Falcon 9 first stage will land successfully


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/landing_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/crash.gif)


More specifically, the launch records are stored in a HTML table shown below:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/falcon9-launches-wiki.png)


  ## Objectives
Web scrap Falcon 9 launch records with `BeautifulSoup`: 
- Extract a Falcon 9 launch records HTML table from Wikipedia
- Parse the table and convert it into a Pandas data frame


First let's import required packages for this lab


In [1]:
!pip3 install beautifulsoup4
!pip3 install requests

In [2]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

and we will provide some helper functions for you to process web scraped HTML table


In [3]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    


To keep the lab tasks consistent, you will be asked to scrape the data from a snapshot of the  `List of Falcon 9 and Falcon Heavy launches` Wikipage updated on
`9th June 2021`


In [4]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/91.0.4472.124 Safari/537.36"
}

Next, request the HTML page from the above URL and get a `response` object


### TASK 1: Request the Falcon9 Launch Wiki page from its URL


First, let's perform an HTTP GET method to request the Falcon9 Launch HTML page, as an HTTP response.


In [5]:
import requests

# Static Wikipedia snapshot URL
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

# Custom headers to mimic a browser
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/91.0.4472.124 Safari/537.36"
}

# Perform the GET request
response = requests.get(static_url, headers=headers)

# Check the response status
print("Status Code:", response.status_code)

Status Code: 200


Create a `BeautifulSoup` object from the HTML `response`


In [6]:
# Create a BeautifulSoup object from the response content
soup = BeautifulSoup(response.text, 'html.parser')

Print the page title to verify if the `BeautifulSoup` object was created properly 


In [8]:
# Use soup.title attribute
print(soup.title.string)

List of Falcon 9 and Falcon Heavy launches - Wikipedia


### TASK 2: Extract all column/variable names from the HTML table header


Next, we want to collect all relevant column names from the HTML table header


Let's try to find all tables on the wiki page first. If you need to refresh your memory about `BeautifulSoup`, please check the external reference link towards the end of this lab


In [9]:
from bs4 import BeautifulSoup

# Parse the HTML content
soup = BeautifulSoup(response.text, 'html.parser')

# Find all tables with class 'wikitable'
tables = soup.find_all('table', {'class': 'wikitable'})

# Choose the correct launch table (usually the first one)
launch_table = tables[0]  # Adjust index if needed

# Extract column headers
column_names = []
for th in launch_table.find_all('th'):
    col_name = extract_column_from_header(th)
    if col_name:
        column_names.append(col_name)

# Display the extracted column names
print(column_names)

['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']


Starting from the third table is our target table contains the actual launch records.


In [11]:
# Find all tables with class 'wikitable'
tables = soup.find_all('table', {'class': 'wikitable'})

# Select the third table (index 2)
launch_table = tables[2]

# Preview the table structure
df_launches = pd.read_html(str(launch_table))[0]
df_launches.head()

,Flight No.,Date andtime (UTC),"Version,Booster[b]",Launch site,Payload[c],Payload mass,Orbit,Customer,Launchoutcome,Boosterlanding
0,14,"10 January 2015,09:47[67]",F9 v1.1B1012[8],"Cape Canaveral,LC-40",SpaceX CRS-5[68](Dragon C107),"2,395 kg (5,280 lb)[69]",LEO (ISS),NASA (CRS),Success[70],Failure (drone ship)
1,14,"Following second-stage separation, SpaceX atte...","Following second-stage separation, SpaceX atte...","Following second-stage separation, SpaceX atte...","Following second-stage separation, SpaceX atte...","Following second-stage separation, SpaceX atte...","Following second-stage separation, SpaceX atte...","Following second-stage separation, SpaceX atte...","Following second-stage separation, SpaceX atte...","Following second-stage separation, SpaceX atte..."
2,15,"11 February 2015,23:03[72]",F9 v1.1B1013[8],"Cape Canaveral,LC-40",DSCOVR[68][73],"570 kg (1,260 lb)",HEO(Sun–Earth L1 insertion),USAF NASA NOAA,Success,Controlled(ocean)[d]
3,15,First launch under USAF's OSP 3 launch contrac...,First launch under USAF's OSP 3 launch contrac...,First launch under USAF's OSP 3 launch contrac...,First launch under USAF's OSP 3 launch contrac...,First launch under USAF's OSP 3 launch contrac...,First launch under USAF's OSP 3 launch contrac...,First launch under USAF's OSP 3 launch contrac...,First launch under USAF's OSP 3 launch contrac...,First launch under USAF's OSP 3 launch contrac...
4,16,"2 March 2015,03:50[21][76]",F9 v1.1B1014[8],"Cape Canaveral,LC-40",ABS-3A Eutelsat 115 West B[68],"4,159 kg (9,169 lb)",GTO,ABS Eutelsat,Success,No attempt[77]


You should able to see the columns names embedded in the table header elements `<th>` as follows:


```
<tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
```


Next, we just need to iterate through the `<th>` elements and apply the provided `extract_column_from_header()` to extract column name one by one


In [12]:
# Select the third table (index 2)
launch_table = tables[2]

# Extract and clean column names
column_names = []
for th in launch_table.find_all('th'):
    col_name = extract_column_from_header(th)
    if col_name:
        column_names.append(col_name)

# Display the result
print(column_names)

['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']


Check the extracted column names


In [13]:
print(column_names)

['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']


## TASK 3: Create a data frame by parsing the launch HTML tables


We will create an empty dictionary with keys from the extracted column names in the previous task. Later, this dictionary will be converted into a Pandas dataframe


In [14]:
launch_dict= dict.fromkeys(column_names)

# Remove an irrelvant column
del launch_dict['Date and time ( )']

# Let's initial the launch_dict with each value to be an empty list
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []
# Added some new columns
launch_dict['Version Booster']=[]
launch_dict['Booster landing']=[]
launch_dict['Date']=[]
launch_dict['Time']=[]

Next, we just need to fill up the `launch_dict` with launch records extracted from table rows.


Usually, HTML tables in Wiki pages are likely to contain unexpected annotations and other types of noises, such as reference links `B0004.1[8]`, missing values `N/A [e]`, inconsistent formatting, etc.


To simplify the parsing process, we have provided an incomplete code snippet below to help you to fill up the `launch_dict`. Please complete the following code snippet with TODOs or you can choose to write your own logic to parse all launch tables:


In [21]:
# Initialize the dictionary if not already done
launch_dict = {key: [] for key in column_names}

# Loop through each row in the launch table (skip header)
for row in launch_table.find_all('tr')[1:]:
    cells = row.find_all('td')
    
    # Skip rows that don't have enough cells
    if len(cells) < 10:
        continue

    # Extract and clean each field using helper functions
    try:
        # Flight No.
        flight_number = cells[0].text.strip()
        launch_dict['Flight No.'].append(flight_number)

        # Date and time (UTC)
        date, time = date_time(cells[1])
        launch_dict['Date and time (UTC)'].append(f"{date} {time}")

        # Version, Booster
        booster = booster_version(cells[2])
        launch_dict['Version, Booster'].append(booster)

        # Launch site
        site = cells[3].text.strip()
        launch_dict['Launch site'].append(site)

        # Payload
        payload = cells[4].text.strip()
        launch_dict['Payload'].append(payload)

        # Payload mass
        mass = get_mass(cells[5])
        launch_dict['Payload mass'].append(mass)

        # Orbit
        orbit = cells[6].text.strip()
        launch_dict['Orbit'].append(orbit)

        # Customer
        customer = cells[7].text.strip()
        launch_dict['Customer'].append(customer)

        # Launch outcome
        outcome = cells[8].text.strip()
        launch_dict['Launch outcome'].append(outcome)

        # Booster landing
        landing = landing_status(cells[9])
        launch_dict['Booster landing'].append(landing)

    except Exception as e:
        print(f"Skipping row due to error: {e}")
        continue

After you have fill in the parsed launch record values into `launch_dict`, you can create a dataframe from it.


In [20]:
df = pd.DataFrame({ key: pd.Series(value, dtype="object") for key, value in launch_dict.items() })

We can now export it to a <b>CSV</b> for the next section, but to make the answers consistent and in case you have difficulties finishing this lab. 

Following labs will be using a provided dataset to make each lab independent. 


<code>df.to_csv('spacex_web_scraped.csv', index=False)</code>


## Authors


<a href="https://www.linkedin.com/in/yan-luo-96288783/">Yan Luo</a>


<a href="https://www.linkedin.com/in/nayefaboutayoun/">Nayef Abou Tayoun</a>


<!--
## Change Log
-->


<!--
| Date (YYYY-MM-DD) | Version | Changed By | Change Description      |
| ----------------- | ------- | ---------- | ----------------------- |
| 2021-06-09        | 1.0     | Yan Luo    | Tasks updates           |
| 2020-11-10        | 1.0     | Nayef      | Created the initial version |
-->


Copyright © 2021 IBM Corporation. All rights reserved.
